In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import os
import pickle


In [16]:
def get_all_file(idprov, band):
    path_prov = '/data/ksa/03_Sampling/data-wide/'+idprov+'/'
    all_files_ = glob(path_prov+'*.pkl')
    files_ = [s for s in all_files_ if band in s]
    return files_

def get_all_file2(idprov, band):
    path_prov = '/data/ksa/04_Data_Preprocessing/'+idprov+'/'
    files_ = glob(path_prov+'*.pkl')
    # files_ = [s for s in all_files_ if band in s]
    return files_
    
def combine_all_data(band, idprov):
    files = get_all_file(idprov, band)
    df_sample_all = pd.DataFrame()
    for i in files:
        with open(i, 'rb') as f:
            df_tmp = pickle.load(f)
        df_sample_all = pd.concat([df_sample_all, df_tmp])
    return df_sample_all

def split_data(df, cutoff_date):
    df['year'] = df['periode'].str[:4]
    df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['nth'].astype(str), format='%Y-%m')

    # Split the dataset into training and testing sets
    training_set = df[df['date'] <= cutoff_date]
    testing_set = df[df['date'] > cutoff_date]

    return training_set, testing_set

In [ ]:
band = 'VV'
idprov='32'
cutoff_date = '2023-09-30'
df_all = combine_all_data(band, idprov)
df_train, df_test = split_data(df_all, '2023-09-30')

In [ ]:
print(f'Training set: {len(df_train)} records')
print(f'Testing set: {len(df_test)} records')
print('Testing set proportion: ', str(round(100*(len(df_test))/(len(df_train)+len(df_test)) ,2)))

In [ ]:
with open('/data/ksa/04_Data_Preprocessing/training-test/32/training_'+idprov+'_'+band+'.pkl', 'wb') as f:
    pickle.dump(df_train, f)

with open('/data/ksa/04_Data_Preprocessing/training-test/32/testing_'+idprov+'_'+band+'.pkl', 'wb') as f:
    pickle.dump(df_test, f)

In [ ]:
df_train.head()
